### Euribor bootstrap and roundtrip validation

After having successfully bootstrapped an OIS curve we can use this curve to build up an EURIBOR curve.

First we copy over what we did in notebook ``OIS Bootstrap`` to build our discount curve

In [1]:
# initiating

import QuantLib as ql

today = ql.Date(10, ql.May, 2019)
ql.Settings.instance().evaluationDate = today

In [2]:
# OIS Par Quotes used as input

ois_quotes = {ql.Period(2, ql.Weeks): ql.SimpleQuote(-0.36503/100),
               ql.Period(1, ql.Months): ql.SimpleQuote(-0.3650/100),
               ql.Period(3, ql.Months): ql.SimpleQuote(-0.36521/100),
               ql.Period(6, ql.Months): ql.SimpleQuote(-0.36754/100),
               ql.Period(1, ql.Years): ql.SimpleQuote(-0.37488/100),
               ql.Period(2, ql.Years): ql.SimpleQuote(-0.36102/100),
               ql.Period(3, ql.Years): ql.SimpleQuote(-0.31511/100),
               ql.Period(5, ql.Years): ql.SimpleQuote(-0.17438/100),
               ql.Period(10, ql.Years): ql.SimpleQuote(0.28879/100),
               ql.Period(15, ql.Years): ql.SimpleQuote(0.62908/100),
               ql.Period(20, ql.Years): ql.SimpleQuote(0.82255/100),
               ql.Period(30, ql.Years): ql.SimpleQuote(0.92238/100)}

In [3]:
eonia_index = ql.Eonia()

ois_helpers = []
for tenor, quote in ois_quotes.items():
    ois_helpers.append(ql.OISRateHelper(0, tenor, ql.QuoteHandle(quote), eonia_index))

eonia_curve = ql.PiecewiseLogCubicDiscount(0, ql.TARGET(), ois_helpers, ql.Actual360())
eonia_curve.enableExtrapolation()

Now, whenever we need a Euro discount curve, we can work with ``ql.YieldTermStructureHandle(eonia_curve)``

Next we take inspiration from the EURIBOR curve bootstrapping section of the Python cookbook. We are building the 6M EURIBOR curve and therefore its first point is the 6M point. This is a Money Market Quote. Afterwards, from 7M to 11M FRAs would be used to build the curve, but to 7 up to the 11M point are not needed for ISDA-SIMM therefore we skip right to the Swap quotes which are starting at 1Y and going up to 30Years.

In [4]:
euribor_money_market = ql.SimpleQuote(-0.231/100)

euribor_helpers = []

euribor_helpers += [ql.DepositRateHelper(ql.QuoteHandle(euribor_money_market), ql.Period(6, ql.Months),2, ql.TARGET(), ql.ModifiedFollowing, False, ql.Actual360())]

euribor_swap_quotes = {ql.Period(1, ql.Years): ql.SimpleQuote(-0.2355/100),
                  ql.Period(2, ql.Years): ql.SimpleQuote(-0.20860/100),
                  ql.Period(3, ql.Years): ql.SimpleQuote(-0.1525/100),
                  ql.Period(5, ql.Years): ql.SimpleQuote(-0.17438/100),
                  ql.Period(10, ql.Years): ql.SimpleQuote(0.28879/100),
                  ql.Period(15, ql.Years): ql.SimpleQuote(0.62908/100),
                  ql.Period(20, ql.Years): ql.SimpleQuote(0.82255/100),
                  ql.Period(30, ql.Years): ql.SimpleQuote(0.92238/100)}

In [5]:
for tenor, quote in euribor_swap_quotes.items():
    euribor_helpers.append(ql.SwapRateHelper(ql.QuoteHandle(quote),
                                             tenor,
                                             ql.TARGET(),
                                             ql.Annual,
                                             ql.Following,
                                             ql.Actual360(),
                                             ql.Euribor6M(),
                                             ql.QuoteHandle(),
                                             ql.Period(0, ql.Days),
                                             ql.YieldTermStructureHandle(eonia_curve)))

In [6]:
euribor6m_curve = ql.PiecewiseLogCubicDiscount(2, ql.TARGET(), euribor_helpers, ql.Actual360())
euribor6m_curve.enableExtrapolation()

In [7]:
pricing_engine = ql.DiscountingSwapEngine(ql.YieldTermStructureHandle(eonia_curve))

In [8]:
reference_swap = euribor_helpers[3].swap()
for cashflow in reference_swap.floatingLeg():
    print(cashflow.date())

for cashflow in reference_swap.fixedLeg():
    print(cashflow.date())

November 14th, 2019
May 14th, 2020
November 16th, 2020
May 14th, 2021
November 15th, 2021
May 16th, 2022
May 14th, 2020
May 14th, 2021
May 16th, 2022


In [9]:
nominal = 1
fixedRate = 0
dc = ql.Actual360()
type = ql.VanillaSwap.Receiver
index = ql.Euribor6M(ql.YieldTermStructureHandle(euribor6m_curve))
float_schedule = ql.Schedule(ql.TARGET().advance(today, 2, ql.Days), ql.Date(16, ql.May, 2022), ql.Period(6, ql.Months), ql.TARGET(), ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Forward, False)
fixed_schedule = ql.Schedule(ql.TARGET().advance(today, 2, ql.Days), ql.Date(16, ql.May, 2022), ql.Period(1, ql.Years), ql.TARGET(), ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Forward, False)

In [10]:
test_swap = ql.VanillaSwap(type, nominal, fixed_schedule, fixedRate, ql.Actual360(), float_schedule, index, 0, ql.Actual360())

In [11]:
for cashflow in test_swap.floatingLeg():
    print(cashflow.date())

for cashflow in test_swap.fixedLeg():
    print(cashflow.date())

test_swap.setPricingEngine(pricing_engine)

November 14th, 2019
May 14th, 2020
November 16th, 2020
May 14th, 2021
November 15th, 2021
May 16th, 2022
May 14th, 2020
May 14th, 2021
May 16th, 2022


In [12]:
for cashflow in test_swap.floatingLeg():
    print(cashflow.date())
    print(cashflow.amount())

for cashflow in reference_swap.floatingLeg():
    print(cashflow.date())
    print(cashflow.amount())

November 14th, 2019
-0.11806666666666665
May 14th, 2020
0.1154437098233152
November 16th, 2020
-0.00011076951453814488
May 14th, 2021
-0.0017318578715532773
November 15th, 2021
-0.0003691437630104488
May 16th, 2022
-5.1282128125662574e-05
November 14th, 2019
-0.11806666666666665
May 14th, 2020
0.1154437098233152
November 16th, 2020
-0.00011076951453814488
May 14th, 2021
-0.0017318578715532773
November 15th, 2021
-0.0003691437630104488
May 16th, 2022
-5.1282128125662574e-05


In [13]:
for cashflow in test_swap.fixedLeg():
    print(cashflow.date())
    print(cashflow.amount())

for cashflow in reference_swap.fixedLeg():
    print(cashflow.date())
    print(cashflow.amount())

May 14th, 2020
0.0
May 14th, 2021
0.0
May 16th, 2022
0.0
May 14th, 2020
0.0
May 14th, 2021
0.0
May 16th, 2022
0.0


In [14]:
print(test_swap.fairRate())
print(euribor_swap_quotes[ql.Period(3, ql.Years)].value())
print(reference_swap.fairRate())

-0.0015249999999999574
-0.0015249999999999999
-0.0015249999999999574


Roundtrip pricing worked out. Needed to adjust Business day convention and Day Count Convention of the SwapHelper to match.
